In [ ]:
from KGS import * # Utilizing my own package for data analysis. Remember to also send enviroment when sharing. 
import scipp as sc
from IPython.display import Image
#from TubeMeasurment_mccode import BIFROST_measurement
mpl.rcParams['font.size']        = 16
import ipywidgets as ipw


In [2]:
#BIFROST_measurement.load_scan('BIFROST_PSC_0p3ms_magnon_v3','Magnon_v3_0p3ms_reduced',folder_path='./', set_Elim=(-0.5,9))

## Organize in scipp array

In [3]:
settings = np.loadtxt('Magnon_v3_0p3ms_reduced').T


In [4]:
qx = sc.array(dims=["row"], values=settings[3], unit="Å^(-1)")
qy = sc.array(dims=["row"], values=settings[4], unit="Å^(-1)")
deltaE = sc.array(dims=["row"], values=settings[2], unit="meV")

recording = sc.DataArray(
    data=sc.array(dims=["row"], values=settings[0], unit="arb. units", variances=settings[1]**2),  #, variances=settings[1]
    coords={"qx": qx, "qy": qy, "deltaE":deltaE}
)
recording

<scipp.DataArray>
Dimensions: Sizes[row:52737756, ]
Coordinates:
* deltaE                    float64            [meV]  (row)  [8.96664, 8.89343, ..., -0.481309, -0.494836]
* qx                        float64           [1/Å]  (row)  [1.10762, 1.10081, ..., 0.297176, 0.295412]
* qy                        float64           [1/Å]  (row)  [0.493428, 0.493309, ..., 2.16931, 2.16798]
Data:
                            float64      [arb. unit]  (row)  [0, 0, ..., 0, 0]  [0, 0, ..., 0, 0]

In [5]:
sc.show(recording)

In [6]:
qx_bins = round((max(recording.coords['qx'].values) - min(recording.coords['qx'].values))/0.04)
qy_bins = round((max(recording.coords['qy'].values) - min(recording.coords['qy'].values))/0.04)
E_bins = round((max(recording.coords['deltaE'].values) - min(recording.coords['deltaE'].values))/0.1)

print(qx_bins, qy_bins, E_bins)
print(max(recording.coords['qx'].values))

112 87 95
3.304249


In [ ]:
da = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced = da.bins.mean()
#sliced.values[(da.bins.size() == sc.index(0)).values] = np.nan
sliced.save_hdf5(filename='Magnon_v3_0p3ms.hdf5')

sc.show(da)

/Users/kristinekrighaar/opt/anaconda3/lib/python3.9/site-packages/scipp/core/deprecation.py:7: VisibleDeprecationWarning: sc.DataArray.attrs has been deprecated and will be removed in Scipp v24.12.0. The deprecation includes sc.DataArray.meta and sc.DataArray.drop_attrs. For unaligned coords, use sc.DataArray.coords and unset the alignment flag. For other attributes, use a higher-level data structure.
  warnings.warn(


## qx,qy plane cuts

In [ ]:

qx_min = 0 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

qy_min = -2 * sc.Unit('Å^(-1)')
qy_max = 4.0 * sc.Unit('Å^(-1)')


def update_plot(amp):
    E_layer = amp * sc.Unit('meV')
    plt.figure(figsize=(6,4))
    p1 = sliced['deltaE', E_layer]['qy', qy_min:qy_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['qy'].values, p1.values, cmap='jet',  vmin=0, vmax=1.0) # vmin=0, vmax=3e-5
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,3)
    plt.ylim(0,4)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$q_y$ [1/Å]')
    plt.title(f"$\Delta E =$ {amp:.2f} meV")
    #plt.savefig('Pb_phonon_BIFROST_qxqy_mid.pdf', format='pdf', bbox_inches='tight')
    

w = ipw.FloatSlider( 
    value=6.19,
    min=min(sliced.coords['deltaE'].values),
    max=9.0,
    step=(sliced.coords['deltaE'].values[1]-sliced.coords['deltaE'].values[0]),
    description="Delta_E:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider

ipw.interactive(update_plot, amp=w)

interactive(children=(FloatSlider(value=6.19, description='Delta_E:', max=9.0, min=-0.499966, step=0.099906431…

## Cut in the qx plane

In [ ]:


qxE = recording.bin(qx=qx_bins, qy=qy_bins, deltaE=E_bins)
sliced_qxE = qxE.bins.mean()

qx_min = -2 * sc.Unit('Å^(-1)')
qx_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 9.0 * sc.Unit('meV')


def update_plot_qxE(amp):
    qy_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qy', qy_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qx'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=1.0) # norm=colors.LogNorm()
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-2,2.5)
    plt.ylim(0,9)
    plt.xlabel('$q_x$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_y =$ {amp:.2f} Å$^{-1}$")
    #plt.savefig('Pb_phonon_BIFROST_qxE_high.pdf', format='pdf', bbox_inches='tight')
    

w_qxE = ipw.FloatSlider( 
    value=2.73,
    min=min(sliced_qxE.coords['qy'].values),
    max=4,
    step=(sliced_qxE.coords['qy'].values[1]-sliced_qxE.coords['qy'].values[0]),
    description="q_y:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider



ipw.interactive(update_plot_qxE, amp=w_qxE)

interactive(children=(FloatSlider(value=2.73, description='q_y:', max=4.0, min=-0.156238, step=0.0398061149425…

In [ ]:
qy_min = -3 * sc.Unit('Å^(-1)')
qy_max = 2 * sc.Unit('Å^(-1)')

E_min = -0.05 * sc.Unit('meV')
E_max = 9.0 * sc.Unit('meV')


def update_plot_qyE(amp):
    qx_layer = amp * sc.Unit('Å^(-1)')
    plt.figure(figsize=(6,4))
    p1 = sliced_qxE['qx', qx_layer]['deltaE', E_min:E_max].transpose()#.plot(cmap='jet')
    im = plt.pcolormesh(p1.coords['qy'].values, p1.coords['deltaE'].values, p1.values, cmap='jet', vmin=0, vmax=1.0)
    cbar = plt.colorbar(im)
    cbar.set_label('I [a.u.]')
    plt.xlim(-1,3)
    plt.ylim(0,7)
    plt.xlabel('$q_y$ [1/Å]')
    plt.ylabel('$\Delta E$ [meV]')
    plt.title(f"$q_x =$ {amp:.2f} Å$^{-1}$")
    #plt.savefig('Pb_phonon_BIFROST_qyE_low.pdf', format='pdf', bbox_inches='tight')
    

w_qyE = ipw.FloatSlider( 
    value=0.17,
    min=min(sliced_qxE.coords['qx'].values),
    max=4,
    step=(sliced_qxE.coords['qx'].values[1]-sliced_qxE.coords['qx'].values[0]),
    description="q_x:",
    disabled=False,
    continuous_update=True,
    orientation="horizontal",
    readout=True,
    readout_format=".2f",
)
# Display the slider


ipw.interactive(update_plot_qyE, amp=w_qyE)

interactive(children=(FloatSlider(value=0.17, description='q_x:', max=4.0, min=-1.171966, step=0.0399662053571…